<h1><center>Data Analytics on Player Performance in Major League Baseball</center></h1>
<h2><center>Chris Emm</center></h2>

In [94]:
import sqlite3
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import matplotlib.ticker as mticker
import requests
from bs4 import BeautifulSoup

pd.set_option('display.max_columns', None)

### Introduction

In order to successfully run a Major League Baseball team, you need to have talented players. Over the years, the definition of what makes players talented has been fluid.

Is it a player who hits a lot of homeruns but strikes out a lot?
<br />Is it a player who might not hit the cover off the ball but fields every play perfectly?
<br />Is it a guy who just gets on base a ton?

The true answer, is all of the above. Every team is going to need people who get on base, people who can drive in runs with one swing of the bat, and people who are great defenders. Now, there are certainly plenty of players who fit within these definitions, however, they normally don't come at a bargain. The goal of this analysis is to determine the *'diamond in the rough type players'* -- the players who are sneakily talented and yet are undervalued, and underpaid. This project will walk you through what makes a player valuable, and how that player's talents can relate to putting up wins for the team he is on, all while identifying the players who came at a bargain based on the talent they brought to the field.

### Part I: Scraping Position Player Data for 2011-2021 Seasons

First, we will need to scrape data from multiple different sources.

**Sean Lahman's Batting Dataset:**
> Contains batting data through the 2021 season.

**Sean Lahman's People Dataset:**
> Contains player information such as birth year, name, etc.

**Sean Lahman's Appearances Dataset:**
> Contains data on how many games a player played at each position throughout the season.
<br />We will use this to remove the pitchers from the batting dataset.

**Sean Lahman's Salary Dataset:**
> Contains player salaries for each year up through 2016. For 2016-Present, we will list it as NaN.


In [103]:
# Create dataframe of player information like name and age
player_table = pd.read_csv('tables/People.csv')
shortened_player = player_table[['playerID','nameFirst', 'nameLast', 'birthYear']]
shortened_player = shortened_player.assign(Name = shortened_player.nameFirst.str.cat(shortened_player.nameLast,sep=' '))
shortened_player = shortened_player[['playerID', 'Name', 'birthYear']]

# Create dataframe of batting statistics
batting_table = pd.read_csv('tables/Batting.csv')
batting_table = batting_table[batting_table.yearID > 2010]

# Combine the pitching and batting stats
batting_table = pd.merge(batting_table, shortened_player, on='playerID')

# Rename columns
batting_table = batting_table.rename(columns={'teamID':'Team', 'lgID':'Lg', 'yearID':'Year'})

# Grabbing only needed columns
batting_table = batting_table[['playerID', 'Name', 'Year','Team', 'Lg', 'G', 'AB', 'R', 'H', '2B', '3B', 'HR', 'RBI', 'SB', \
                             'CS', 'BB', 'SO', 'IBB', 'HBP', 'SH', 'SF', 'GIDP']]

# Changing columns to be ints instead of floats
#full_batting.Age = full_batting.Age.astype(int)
batting_table.RBI = batting_table.RBI.astype(int)
batting_table.SB = batting_table.SB.astype(int)
batting_table.CS = batting_table.CS.astype(int)
batting_table.SO = batting_table.SO.astype(int)
batting_table.IBB = batting_table.IBB.astype(int)
batting_table.HBP = batting_table.HBP.astype(int)
batting_table.SH = batting_table.SH.astype(int)
batting_table.SF = batting_table.SF.astype(int)
batting_table.GIDP = batting_table.GIDP.astype(int)
                          
# Sort by year and then reset the index
batting_table = batting_table.sort_values('Year')
batting_table = batting_table.reset_index(drop=True)

batting_table

,playerID,Name,Year,Team,Lg,G,AB,R,H,2B,3B,HR,RBI,SB,CS,BB,SO,IBB,HBP,SH,SF,GIDP
0,abadfe01,Fernando Abad,2011,HOU,NL,29,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,fisheca01,Carlos Fisher,2011,CIN,NL,17,2,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
2,figuene01,Nelson Figueroa,2011,HOU,NL,8,9,0,2,0,0,0,0,0,0,0,2,0,0,1,0,0
3,figgich01,Chone Figgins,2011,SEA,AL,81,288,24,54,11,1,1,15,11,6,21,42,1,0,2,2,6
4,salech01,Chris Sale,2011,CHA,AL,58,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16269,flaheja01,Jack Flaherty,2021,SLN,NL,17,17,2,2,0,0,1,3,0,0,5,7,0,0,3,1,0
16270,flexech01,Chris Flexen,2021,SEA,AL,31,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
16271,fowledu01,Dustin Fowler,2021,PIT,NL,18,41,3,7,1,0,0,2,1,0,3,20,0,1,0,1,0
16272,farmeky01,Kyle Farmer,2021,CIN,NL,147,483,60,127,22,2,16,63,2,3,22,97,1,18,1,5,16


However, because pitchers can also bat (and they were required to in the National League until 2022), this dataset includes pitchers. For example, Jack Flaherty is a starting pitcher for the St. Louis Cardinals, and he is included in the dataset. However, we want to remove these pitchers and focus only on true position players batting. We will take care of this is the next section.

#### Removing Pitchers from Batting Table (Except for Shohei Ohtani)

Since pitchers can hit too (and they are required to in the National League), the Lahman Batting dataset included pitchers too. However, most of the time, pitchers are poor hitters, so I did not want to include them, as it would skew the dataset. In the below code snippet, I am removing all rows of players who have less than 5 appearances at any position that is not pitcher.

In [104]:
appearances = pd.read_csv('tables/Appearances.csv')
appearances = appearances.assign(Games_Not_Pitching = appearances.G_c + appearances.G_1b + appearances.G_2b + \
                                 appearances.G_3b + appearances.G_ss + appearances.G_of + appearances.G_dh)

appearances = appearances[['playerID', 'yearID', 'Games_Not_Pitching']]
appearances = appearances.rename(columns={'yearID':'Year'})


# Merge
batting_table = pd.merge(batting_table, appearances, on=['playerID', 'Year'])

batting_table = batting_table[batting_table.Games_Not_Pitching > 5]
batting_table = batting_table[['Name', 'Year','Team', 'Lg', 'G', 'AB', 'R', 'H', '2B', '3B', 'HR', 'RBI', 'SB', \
                             'CS', 'BB', 'SO', 'IBB', 'HBP', 'SH', 'SF', 'GIDP']]
batting_table

,Name,Year,Team,Lg,G,AB,R,H,2B,3B,HR,RBI,SB,CS,BB,SO,IBB,HBP,SH,SF,GIDP
3,Chone Figgins,2011,SEA,AL,81,288,24,54,11,1,1,15,11,6,21,42,1,0,2,2,6
5,Jarrod Saltalamacchia,2011,BOS,AL,103,358,52,84,23,3,16,56,1,0,24,119,1,3,0,1,7
8,Thomas Field,2011,COL,NL,16,48,4,13,0,0,0,3,0,0,3,14,0,0,0,0,1
11,Prince Fielder,2011,MIL,NL,162,569,95,170,36,1,38,120,1,1,107,106,32,10,0,6,17
12,Angel Sanchez,2011,HOU,NL,110,288,35,69,10,0,1,28,3,0,27,44,1,1,10,2,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19266,Adam Engel,2021,CHA,AL,39,123,21,31,9,0,7,18,7,1,11,31,0,5,0,1,4
19269,Phillip Evans,2021,PIT,NL,76,214,23,44,5,0,5,16,1,0,28,53,1,5,0,0,3
19270,Rhys Hoskins,2021,PHI,NL,107,389,64,96,29,0,27,71,3,2,47,108,0,5,0,2,7
19277,Dustin Fowler,2021,PIT,NL,18,41,3,7,1,0,0,2,1,0,3,20,0,1,0,1,0


As you can see, Jack Flaherty is no longer in the dataset, so it appears we have successfully removed pitchers from the dataset. Next, we want to consider the metrics that better define player value. There are a few statistics that are necessary to consider when discussing a player's value.

<br />**WAR: Wins Above Replacement**
> Calculates many more wins a baseball team has from a player compared to if the team replaced the player with a replacement-level player in the same position.

<br />**OPS+: Normalized on-base plus slugging percentage**
> **On-Base Percentage:** The ratio of the sum of the batter's hits, walks, hit by pitches to their number of plate appearances
<br />**Slugging Percentage:** The total number of bases a player records per at-bat **(1B + 2Bx2 + 3Bx3 + HRx4)/AB**
<br />**Normalized:** Adjusts the OPS based on park factors by comparing it to league average **(OPS / league OPS) x 100**

In [105]:
# Scrape data from baseball reference .csv file
baseball_reference_table = pd.read_csv('baseball-reference/batting.csv')
baseball_reference_table = baseball_reference_table[['Name', 'Age', 'Year', 'Team', 'Lg', 'WAR', 'WAR_def', 'WAR_off', 'OPS_plus']]

# Rename OPS_plus to OPS+
baseball_reference_table = baseball_reference_table.rename(columns={'OPS_plus':'OPS+'})

# Cast OPS+ column to integer
baseball_reference_table['OPS+'] = baseball_reference_table['OPS+'].fillna(0)
baseball_reference_table['OPS+'] = baseball_reference_table['OPS+'].astype(int)

# Cast Age column to integer
baseball_reference_table.Age = baseball_reference_table.Age.fillna(0)
baseball_reference_table.Age = baseball_reference_table.Age.astype(int)

# Merge dataframe from above with the nex dataframe
batting_table = pd.merge(batting_table, baseball_reference_table, on=['Name', 'Year', 'Team', 'Lg'])

# Reorder the columns
batting_table = batting_table[['Year', 'Name', 'Age', 'Team', 'Lg', 'G', 'AB', 'R', 'H', '2B', '3B', 'HR', 'RBI', 'SB', 'CS', \
               'BB', 'SO', 'IBB', 'HBP', 'SH', 'SF', 'GIDP', 'OPS+', 'WAR', 'WAR_def', 'WAR_off']]

batting_table = batting_table.drop_duplicates()

In [106]:
batting_table

,Year,Name,Age,Team,Lg,G,AB,R,H,2B,3B,HR,RBI,SB,CS,BB,SO,IBB,HBP,SH,SF,GIDP,OPS+,WAR,WAR_def,WAR_off
0,2011,Chone Figgins,33,SEA,AL,81,288,24,54,11,1,1,15,11,6,21,42,1,0,2,2,6,40,-0.92,0.06,-0.78
1,2011,Jarrod Saltalamacchia,26,BOS,AL,103,358,52,84,23,3,16,56,1,0,24,119,1,3,0,1,7,94,0.73,-0.05,1.51
2,2011,Thomas Field,24,COL,NL,16,48,4,13,0,0,0,3,0,0,3,14,0,0,0,0,1,51,0.20,0.40,-0.14
3,2011,Prince Fielder,27,MIL,NL,162,569,95,170,36,1,38,120,1,1,107,106,32,10,0,6,17,164,4.49,-2.12,5.53
4,2011,Angel Sanchez,27,HOU,NL,110,288,35,69,10,0,1,28,3,0,27,44,1,1,10,2,3,65,-0.31,0.12,-0.11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4904,2021,Mitch Garver,30,MIN,AL,68,207,29,53,15,0,13,34,1,1,31,71,0,3,0,2,4,139,2.07,0.27,2.15
4905,2021,Phillip Evans,28,PIT,NL,76,214,23,44,5,0,5,16,1,0,28,53,1,5,0,0,3,68,-0.37,-0.37,-0.24
4906,2021,Rhys Hoskins,28,PHI,NL,107,389,64,96,29,0,27,71,3,2,47,108,0,5,0,2,7,129,1.95,-1.32,2.70
4907,2021,Dustin Fowler,26,PIT,NL,18,41,3,7,1,0,0,2,1,0,3,20,0,1,0,1,0,20,-0.36,0.01,-0.36


#### Scrape Salaries for Players from 2011-2021

Having a discussion about player value would be worthless if we did not include the salaries for each player for every year. Unfortunately, the updated (to the end of the 2021 season) Lahman dataset did not have salaries for any of the players, nor did the baseball reference .csv file. Instead, I had to do some research and found that Cot's Baseball Contracts (https://legacy.baseballprospectus.com/compensation/cots/) had salaries for all players from 2000-2021. There were individual .csv files for each season, so using excel, I just merged them all into one table in one file. I then uploaded that file to my workspace and read it as normal (using the read_csv() function). Below is a little bit of data manipulation; just trying to get the format of the names to be the exact format that my dataframe had, so that it would smoothly merge into my dataframe.

In [107]:
salaries = pd.read_csv('salaries/salaries.csv')

# Split the Name colum into first name and last name (originally stored as 'Last, First')
# and store it as a separate dataframe
names = salaries.Player.str.split(', ', expand=True)[[0, 1]]

# Create a new column called Name that has the format 'First Last'
names = names.assign(Name = names[1].str.cat(names[0],sep=' '))

# Remove all columns except for the new name column
names = names[['Name']]

# Add the years to the names dataframe
names = names.assign(Year = salaries.Year.to_list())

# Add the salaries to the names dataframe
names = names.assign(Salary = salaries.Salary.to_list())

# Assign names to the salaries variable
salaries = names

salaries

,Name,Year,Salary
0,Mike Trout,2021,"$37,116,667"
1,Gerrit Cole,2021,"$36,000,000"
2,Jacob deGrom,2021,"$36,000,000"
3,Stephen Strasburg,2021,"$35,000,000"
4,Nolan Arenado,2021,"$35,000,000"
...,...,...,...
11080,Brayan Villarreal,2011,"$414,000"
11081,Jordan Walden,2011,"$414,000"
11082,Ryan Webb,2011,"$414,000"
11083,Tom Wilhelmsen,2011,"$414,000"


#### Add Salaries to Batting Statistics Dataframe

Now that we have salaries for every single player in the last decade, we can merge the salary data onto the table that we have that contains the batting statistics for every single player from 2011-2021. Below is the code that achieves this.

In [108]:
batting_table = pd.merge(batting_table, salaries, on = ['Name', 'Year'])
batting_table

,Year,Name,Age,Team,Lg,G,AB,R,H,2B,3B,HR,RBI,SB,CS,BB,SO,IBB,HBP,SH,SF,GIDP,OPS+,WAR,WAR_def,WAR_off,Salary
0,2011,Chone Figgins,33,SEA,AL,81,288,24,54,11,1,1,15,11,6,21,42,1,0,2,2,6,40,-0.92,0.06,-0.78,"$9,500,000"
1,2011,Jarrod Saltalamacchia,26,BOS,AL,103,358,52,84,23,3,16,56,1,0,24,119,1,3,0,1,7,94,0.73,-0.05,1.51,"$750,000"
2,2011,Prince Fielder,27,MIL,NL,162,569,95,170,36,1,38,120,1,1,107,106,32,10,0,6,17,164,4.49,-2.12,5.53,"$15,500,000"
3,2011,Angel Sanchez,27,HOU,NL,110,288,35,69,10,0,1,28,3,0,27,44,1,1,10,2,3,65,-0.31,0.12,-0.11,"$432,500"
4,2011,Carlos Santana,25,CLE,AL,155,552,84,132,35,2,27,79,5,3,97,133,7,2,0,7,15,126,4.14,-0.03,4.52,"$416,600"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3112,2021,Mitch Garver,30,MIN,AL,68,207,29,53,15,0,13,34,1,1,31,71,0,3,0,2,4,139,2.07,0.27,2.15,"$1,875,000"
3113,2021,Phillip Evans,28,PIT,NL,76,214,23,44,5,0,5,16,1,0,28,53,1,5,0,0,3,68,-0.37,-0.37,-0.24,"$650,000"
3114,2021,Rhys Hoskins,28,PHI,NL,107,389,64,96,29,0,27,71,3,2,47,108,0,5,0,2,7,129,1.95,-1.32,2.70,"$4,800,000"
3115,2021,Dustin Fowler,26,PIT,NL,18,41,3,7,1,0,0,2,1,0,3,20,0,1,0,1,0,20,-0.36,0.01,-0.36,"$575,500"


### Part II: Scraping Position Player Data for 2011-2021 Seasons

In order to successfully analyze a player's value, we need to have more than just the basic metrics like batting average, RBI, and homeruns. We want to more advanced data, such as walk rate, strikeout rate, wRC+, etc. Since our original dataset does not include this, I had to scrape data from Fangraphs. 

Since Fangraphs only has stats by team for a given year, I had to create a function that created a dataframe by scraping the table using the given url, which was specific to a team and year. For example, one url would have the datatable for the 2016 Baltimore Orioles. Since one page could only display 50 players, there were usually 2 pages for each team. Below is the function that creates the dataframe by scraping the data from the inputted url.

In [28]:
def scraping_advanced_stats(url, year):
    # Extracting text from webpage
    html = requests.get(url).text

    # Parsing the text into html code
    soup = BeautifulSoup(html,"html.parser")

    # Finding the table in the html code - we are searching by the id of the table
    table = soup.find("table", attrs={"class": "rgMasterTable"})


    table_data = table.tbody.find_all("tr")

    dataset = []
    for tr in table_data:
        temp = ()
        for td in tr.find_all("td"):
            if '\xa0' in td.text:
                temp += ('0.0',)
            else:
                temp += (td.text,)
        dataset.append(temp)

    advanced_stats = pd.DataFrame(data = dataset)
    advanced_stats = advanced_stats.replace(to_replace=" NULL",value=0)

    table_header = table.thead.find_all("tr")
    columns = []
    count = 0
    for tr in table_header:
        if count == 1:
            th = tr.find_all("th")
            for a in th:
                columns.append(a.text)
        count = 1
    advanced_stats.columns = columns
    advanced_stats = advanced_stats.assign(Year = year)
    
    return advanced_stats

Since I needed to generate a url for each page for each team for a given year for the years 2011-2021, I needed to create a function that generated a url for each value. See it below:

In [39]:
def get_urls(team, year, page):
    url = 'https://www.fangraphs.com/leaders.aspx?pos=all&stats' \
      '=bat&lg=all&qual=0&type=1&season=' + str(year) + '&month=0&season1=' \
       + str(year) + '&ind=0&team='+ str(team) +'&rost=0&age=0&filter=&players=0&startdate' \
      '=' + str(year) + '-01-01&enddate=' + str(year) + '-12-31&page=' + str(page) + '_50'
    return url

Below, I am actually putting the functions made above into use. With the first url, I create an initial dataframe. For the subsequent urls, I just concatenate the created dataframe onto the previous dataframes, to create a single dataframe that holds the data for each player on each team from 2011-2021. Below is the code:

In [45]:
teams = [i for i in range(1, 31)]
years = [i for i in range(2011,2022)]
pages = [1,2]

count = 0
for year in years:
    for team in teams:
        for page in pages:
            url = get_urls(team, year, page)
            if count == 0:
                advanced_batting = scraping_advanced_stats(url, year)
                count = 1
            else:
                advanced_batting = pd.concat([advanced_batting, scraping_advanced_stats(url, year)])
    
advanced_batting = advanced_batting.drop_duplicates()
advanced_batting

,#,Name,PA,BB%,K%,BB/K,AVG,OBP,SLG,OPS,ISO,Spd,BABIP,UBR,wGDP,wSB,wRC,wRAA,wOBA,wRC+,Year
0,1,Tyler Chatwood,5,0.0%,0.0%,0.00,.667,.667,.667,1.333,.000,2.6,.667,0.0,0.0,0.0,2,1.1,.594,289,2011
1,2,Ervin Santana,2,0.0%,50.0%,0.00,.500,.500,.500,1.000,.000,0.1,1.000,0.0,0.0,0.0,0,0.2,.445,188,2011
2,3,Gil Velazquez,7,0.0%,0.0%,0.00,.500,.429,.500,.929,.000,0.1,.429,0.0,0.1,0.0,1,0.4,.382,145,2011
3,4,Howie Kendrick,583,5.7%,20.4%,0.28,.285,.338,.464,.802,.179,6.2,.338,2.3,-2.2,-0.1,81,15.4,.349,123,2011
4,5,Torii Hunter,649,9.6%,19.3%,0.50,.262,.336,.429,.765,.167,3.1,.297,1.3,-3.5,-2.4,84,10.7,.337,115,2011
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49,50,Conner Menez,0,0.0%,0.0%,0.00,.000,.000,.000,.000,.000,0.1,.000,0.0,0.0,0.0,0,0.0,.000,0.0,2021
0,51,Caleb Baragar,2,0.0%,100.0%,0.00,.000,.000,.000,.000,.000,0.1,.000,0.0,0.0,0.0,0,-0.5,.000,-100,2021
1,52,Kervin Castro,0,0.0%,0.0%,0.00,.000,.000,.000,.000,.000,0.1,.000,0.0,0.0,0.0,0,0.0,.000,0.0,2021
2,53,Gregory Santos,0,0.0%,0.0%,0.00,.000,.000,.000,.000,.000,0.1,.000,0.0,0.0,0.0,0,0.0,.000,0.0,2021


Below you will see a naming discrepancy between my first dataframe (first table displayed) and the newly created dataframe (2nd table displayed). We need to address this to be able to merge our advanced data by name.

In [109]:
batting_table[batting_table.Name == 'Cedric Mullins']

,Year,Name,Age,Team,Lg,G,AB,R,H,2B,3B,HR,RBI,SB,CS,BB,SO,IBB,HBP,SH,SF,GIDP,OPS+,WAR,WAR_def,WAR_off,Salary
2258,2019,Cedric Mullins,24,BAL,AL,22,64,7,6,0,2,0,4,1,0,4,14,0,3,2,1,2,-8,-0.54,0.17,-0.65,"$557,500"
2620,2020,Cedric Mullins,25,BAL,AL,48,140,16,38,4,3,3,12,7,2,8,37,0,1,4,0,0,94,0.44,0.05,0.45,"$563,500"
3007,2021,Cedric Mullins,26,BAL,AL,159,602,91,175,37,5,30,59,30,8,59,125,3,8,1,4,2,135,5.69,0.35,5.75,"$577,000"


In [114]:
advanced_batting[advanced_batting.Name == 'Cedric Mullins II']

,#,Name,PA,BB%,K%,BB/K,AVG,OBP,SLG,OPS,ISO,Spd,BABIP,UBR,wGDP,wSB,wRC,wRAA,wOBA,wRC+,Year
11,12,Cedric Mullins II,191,8.9%,19.4%,0.46,.235,.312,.359,.671,.124,2.9,.279,0.8,0.2,-0.9,20,-2.7,.298,86,2018
23,24,Cedric Mullins II,74,5.4%,18.9%,0.29,.094,.181,.156,.337,.063,7.9,.118,0.5,-0.2,0.2,-1,-10.3,.159,-12,2019
9,10,Cedric Mullins II,153,5.2%,24.2%,0.22,.271,.315,.407,.723,.136,7.2,.350,1.8,0.6,0.4,18,-0.9,.313,95,2020
1,2,Cedric Mullins II,675,8.7%,18.5%,0.47,.291,.360,.518,.878,.228,6.1,.322,0.4,2.3,2.1,114,32.0,.372,136,2021


Below, we will address this issue and fix it so that we can successfully merge by name: